In [1]:
from pathlib import Path
from bids import BIDSLayout
from contarg.utils import make_rel_symlink
import json
import subprocess
import nibabel as nb
import scipy.io as sio
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

In [ ]:
subjects = ['24546']

liston_scripts_dir = Path('/data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline')
bids_dir = Path('/data/EDB/TMSpilot/BIDS')
liston_dir = Path('/data/EDB/TMSpilot/liston')
liston_dir.mkdir(exist_ok=True)

overwrite = False

jobids = {}

fd_thresh = 0.3

# make a symlinked liston input directory structure

In [41]:
layout = BIDSLayout(bids_dir)

In [42]:
fmap_dir_map = {
    'j-':'AP',
    'j':'PA'
}

In [43]:
# create input directory
for subject in subjects:
    lsub_dir = liston_dir / f'sub-{subject}'
    lsub_dir.mkdir(exist_ok=True)
    anat_dir = lsub_dir / 'anat'
    t1w_dir = anat_dir / 'unprocessed/T1w'
    t1w_dir.mkdir(exist_ok=True, parents=True)
    t2w_dir = anat_dir / 'unprocessed/T2w'
    t2w_dir.mkdir(exist_ok=True, parents=True)
    t1ws = layout.get(return_type='file', subject=subject, datatype='anat', suffix='T1w', extension='.nii.gz')
    t1ws = [tt for tt in t1ws if not 'norm' in tt]
    for ix, t1w in enumerate(t1ws):
        lpath = t1w_dir /f'T1w_{ix + 1}.nii.gz'
        make_rel_symlink(lpath, Path(t1w))

    t2ws = layout.get(return_type='file', subject=subject, datatype='anat', suffix='T2w', extension='.nii.gz')
    t2ws = [tt for tt in t2ws if not 'norm' in tt]
    for ix, t2w in enumerate(t2ws):
        lpath = t2w_dir /f'T2w_{ix + 1}.nii.gz'
        make_rel_symlink(lpath, Path(t2w))
        
    func_dir = lsub_dir / 'func/unprocessed/rest'
    func_dir.mkdir(exist_ok=True, parents=True)
    fmap_dir = lsub_dir / 'func/unprocessed/field_maps'
    fmap_dir.mkdir(exist_ok=True, parents=True)
    
    # make fmap_dict
    field_maps = layout.get(subject=subject, datatype='fmap', extension='.nii.gz')
    fmap_lut = {}
    for fmap in field_maps:
        fmap_lut[fmap.get_metadata()['B0FieldIdentifier']] = {}
    for fmap in field_maps:
        fmap_lut[fmap.get_metadata()['B0FieldIdentifier']][fmap_dir_map[fmap.get_metadata()['PhaseEncodingDirection']]] = fmap
    for session in layout.get(return_type='id', target='session', subject=subject):
        ses_dir = func_dir/f'session_{session}'
        lrun = 1
        for brun in layout.get(return_type='id', target='run', subject=subject, session=session):
            if (subject == '24704') & (brun == '1') & (session == '1'):
                continue
            me_bolds = layout.get(subject=subject,
                                  session=session,
                                  run=brun,
                                  datatype='func', acquisition='MBME', extension='.nii.gz')
            run_dir = ses_dir / f'run_{lrun}'
            run_dir.mkdir(exist_ok=True, parents=True)
            for bold in me_bolds:
                echo = bold.get_entities()['echo']
                bjson = bold.get_associations()[0]
                lpath = run_dir / f'Rest_S{session}_R{lrun}_E{echo}.nii.gz'
                make_rel_symlink(lpath, Path(bold.path))
                lbjson_path = run_dir / f'Rest_S{session}_R{lrun}_E{echo}.json'
                make_rel_symlink(lbjson_path, Path(bjson.path))

                if echo == '1':
                    fsource = bold.get_metadata()['B0FieldSource']
                    lapfmap_path = fmap_dir / f'AP_S{session}_R{lrun}.nii.gz'
                    make_rel_symlink(lapfmap_path, Path(fmap_lut[fsource]['AP'].path))
                    lapfmapjson_path = fmap_dir / f'AP_S{session}_R{lrun}.json'
                    make_rel_symlink(lapfmapjson_path, Path(fmap_lut[fsource]['AP'].get_associations()[0].path))
                    try:
                        lpafmap_path = fmap_dir / f'PA_S{session}_R{lrun}.nii.gz'
                        make_rel_symlink(lpafmap_path, Path(fmap_lut[fsource]['PA'].path))
                        lpafmapjson_path = fmap_dir / f'PA_S{session}_R{lrun}.json'
                        make_rel_symlink(lpafmapjson_path, Path(fmap_lut[fsource]['PA'].get_associations()[0].path))
                    except KeyError:
                        # if the PA doesn't exist, we'll have to make it
                        cmd = [
                            '3dcalc',
                            '-a',
                            f"{bold.path}[0..9]",
                            '-expr',
                            "a",
                            '-prefix',
                            lpafmap_path.as_posix(),
                            '-overwrite'
                        ]
                        subprocess.run(cmd, check=True)
                        lpafmapjson_path = fmap_dir / f'PA_S{session}_R{lrun}.json'
                        fmap_json_dat = json.loads(lapfmapjson_path.read_text())
                        fmap_json_dat['PhaseEncodingDirection'] = 'j'
                        lpafmapjson_path.write_text(json.dumps(fmap_json_dat, indent=2))
            lrun += 1

In [43]:
# mv sub-24718_ses-3 to a second subject for liston purposes
new_ses_3_dir = liston_dir/'sub-247183'
new_ses_3_dir.mkdir(exist_ok=True)
old_sub_dir =  liston_dir/'sub-24718'

# mv unprocessed rest to new directory
new_rest_dir = new_ses_3_dir / 'func/unprocessed/rest/session_1'
old_rest_dir = old_sub_dir / 'func/unprocessed/rest/session_3'
try:
    new_rest_dir.parent.mkdir(exist_ok=True, parents=True)
    old_rest_dir.rename(new_rest_dir)
except FileNotFoundError:
    pass
# rename rest files
new_rest_dir_oldfiles = sorted(new_rest_dir.glob('*/Rest_*.*'))
for ff in new_rest_dir_oldfiles:
    new_ff = ff.as_posix().replace('_S3_', '_S1_')
    ff.rename(new_ff)
    
# make new fieldmap directory
old_fm_dir = old_sub_dir / 'func/unprocessed/field_maps'
new_fm_dir = new_ses_3_dir / 'func/unprocessed/field_maps'
new_fm_dir.mkdir(exist_ok=True)
# rename fieldmaps
old_s3_fms = sorted(old_fm_dir.glob('*_S3_*.*'))
for ff in old_s3_fms:
    new_ff = new_fm_dir / ff.parts[-1].replace('_S3_', '_S1_')
    ff.rename(new_ff)



In [90]:
# create directories with mbfieldmaps for 247183, 24740, and 24742
altbids_dir = Path('/data/EDB/TMSpilot/BIDS.alternatefmap')

altlayout = BIDSLayout(altbids_dir)

In [100]:
liston_dir = Path('/data/EDB/TMSpilot/liston')


In [93]:
subjects = ['24718', '24740', '24742']
# create input directory
for subject in subjects:
    lsub_dir = liston_dir / f'sub-{subject}sbref'
    lsub_dir.mkdir(exist_ok=True)
    anat_dir = lsub_dir / 'anat'
    t1w_dir = anat_dir / 'unprocessed/T1w'
    t1w_dir.mkdir(exist_ok=True, parents=True)
    t2w_dir = anat_dir / 'unprocessed/T2w'
    t2w_dir.mkdir(exist_ok=True, parents=True)
    t1ws = altlayout.get(return_type='file', subject=subject, datatype='anat', suffix='T1w', extension='.nii.gz')
    t1ws = [tt for tt in t1ws if not 'norm' in tt]
    for ix, t1w in enumerate(t1ws):
        lpath = t1w_dir /f'T1w_{ix + 1}.nii.gz'
        make_rel_symlink(lpath, Path(t1w))

    t2ws = altlayout.get(return_type='file', subject=subject, datatype='anat', suffix='T2w', extension='.nii.gz')
    t2ws = [tt for tt in t2ws if not 'norm' in tt]
    for ix, t2w in enumerate(t2ws):
        lpath = t2w_dir /f'T2w_{ix + 1}.nii.gz'
        make_rel_symlink(lpath, Path(t2w))
        
    func_dir = lsub_dir / 'func/unprocessed/rest'
    func_dir.mkdir(exist_ok=True, parents=True)
    fmap_dir = lsub_dir / 'func/unprocessed/field_maps'
    fmap_dir.mkdir(exist_ok=True, parents=True)
    
    # make fmap_dict
    field_maps = altlayout.get(subject=subject, datatype='fmap', extension='.nii.gz')
    fmap_lut = {}
    for fmap in field_maps:
        fmap_lut[fmap.get_metadata()['B0FieldIdentifier']] = {}
    for fmap in field_maps:
        fmap_lut[fmap.get_metadata()['B0FieldIdentifier']][fmap_dir_map[fmap.get_metadata()['PhaseEncodingDirection']]] = fmap
    for session in altlayout.get(return_type='id', target='session', subject=subject):
        ses_dir = func_dir/f'session_{session}'
        lrun = 1
        for brun in altlayout.get(return_type='id', target='run', subject=subject, session=session):
            if (subject == '24704') & (brun == '1') & (session == '1'):
                continue
            me_bolds = altlayout.get(subject=subject,
                                  session=session,
                                  run=brun,
                                  datatype='func', acquisition='MBME', extension='.nii.gz')
            run_dir = ses_dir / f'run_{lrun}'
            run_dir.mkdir(exist_ok=True, parents=True)
            for bold in me_bolds:
                echo = bold.get_entities()['echo']
                bjson = bold.get_associations()[0]
                lpath = run_dir / f'Rest_S{session}_R{lrun}_E{echo}.nii.gz'
                make_rel_symlink(lpath, Path(bold.path))
                lbjson_path = run_dir / f'Rest_S{session}_R{lrun}_E{echo}.json'
                make_rel_symlink(lbjson_path, Path(bjson.path))

                if echo == '1':
                    fsource = bold.get_metadata()['B0FieldSource']
                    lapfmap_path = fmap_dir / f'AP_S{session}_R{lrun}.nii.gz'
                    make_rel_symlink(lapfmap_path, Path(fmap_lut[fsource]['AP'].path))
                    lapfmapjson_path = fmap_dir / f'AP_S{session}_R{lrun}.json'
                    make_rel_symlink(lapfmapjson_path, Path(fmap_lut[fsource]['AP'].get_associations()[0].path))
                    try:
                        lpafmap_path = fmap_dir / f'PA_S{session}_R{lrun}.nii.gz'
                        make_rel_symlink(lpafmap_path, Path(fmap_lut[fsource]['PA'].path))
                        lpafmapjson_path = fmap_dir / f'PA_S{session}_R{lrun}.json'
                        make_rel_symlink(lpafmapjson_path, Path(fmap_lut[fsource]['PA'].get_associations()[0].path))
                    except KeyError:
                        # if the PA doesn't exist, we'll have to make it
                        cmd = [
                            '3dcalc',
                            '-a',
                            f"{bold.path}[0..9]",
                            '-expr',
                            "a",
                            '-prefix',
                            lpafmap_path.as_posix(),
                            '-overwrite'
                        ]
                        subprocess.run(cmd, check=True)
                        lpafmapjson_path = fmap_dir / f'PA_S{session}_R{lrun}.json'
                        fmap_json_dat = json.loads(lapfmapjson_path.read_text())
                        fmap_json_dat['PhaseEncodingDirection'] = 'j'
                        lpafmapjson_path.write_text(json.dumps(fmap_json_dat, indent=2))
            lrun += 1

++ 3dcalc: AFNI version=AFNI_23.2.08 (Aug 22 2023) [64-bit]
++ Authored by: A cast of thousands
++ WARNING: file /home/nielsond/.afni.log is now 865859908 (866 million) bytes long!
++ Output dataset /data/EDB/TMSpilot/liston/sub-24718sbref/func/unprocessed/field_maps/PA_S1_R1.nii.gz
++ 3dcalc: AFNI version=AFNI_23.2.08 (Aug 22 2023) [64-bit]
++ Authored by: A cast of thousands
++ WARNING: file /home/nielsond/.afni.log is now 865860173 (866 million) bytes long!
++ Output dataset /data/EDB/TMSpilot/liston/sub-24718sbref/func/unprocessed/field_maps/PA_S1_R2.nii.gz
++ 3dcalc: AFNI version=AFNI_23.2.08 (Aug 22 2023) [64-bit]
++ Authored by: A cast of thousands
++ WARNING: file /home/nielsond/.afni.log is now 865860438 (866 million) bytes long!
++ Output dataset /data/EDB/TMSpilot/liston/sub-24718sbref/func/unprocessed/field_maps/PA_S1_R3.nii.gz
++ 3dcalc: AFNI version=AFNI_23.2.08 (Aug 22 2023) [64-bit]
++ Authored by: A cast of thousands
++ WARNING: file /home/nielsond/.afni.log is now 865

In [94]:
# mv sub-24718_ses-3 to a second subject for liston purposes
new_ses_3_dir = liston_dir/'sub-247183sbref'
new_ses_3_dir.mkdir(exist_ok=True)
old_sub_dir =  liston_dir/'sub-24718sbref'

# mv unprocessed rest to new directory
new_rest_dir = new_ses_3_dir / 'func/unprocessed/rest/session_1'
old_rest_dir = old_sub_dir / 'func/unprocessed/rest/session_3'
try:
    new_rest_dir.parent.mkdir(exist_ok=True, parents=True)
    old_rest_dir.rename(new_rest_dir)
except FileNotFoundError:
    pass
# rename rest files
new_rest_dir_oldfiles = sorted(new_rest_dir.glob('*/Rest_*.*'))
for ff in new_rest_dir_oldfiles:
    new_ff = ff.as_posix().replace('_S3_', '_S1_')
    ff.rename(new_ff)
    
# make new fieldmap directory
old_fm_dir = old_sub_dir / 'func/unprocessed/field_maps'
new_fm_dir = new_ses_3_dir / 'func/unprocessed/field_maps'
new_fm_dir.mkdir(exist_ok=True)
# rename fieldmaps
old_s3_fms = sorted(old_fm_dir.glob('*_S3_*.*'))
for ff in old_s3_fms:
    new_ff = new_fm_dir / ff.parts[-1].replace('_S3_', '_S1_')
    ff.rename(new_ff)

# manually copy the anat directory over

# run hires HCP wrapper
This runs Freesurfer along with some additional processing, it can take over 12 hours
example comand:
`./anat_highres_HCP_wrapper_par.sh /data/EDB/TMSpilot/liston sub-24704 10`

In [45]:
anat_script_path = (liston_scripts_dir /'anat_highres_HCP_wrapper_par.sh').as_posix()


swarm_cmd_dir = liston_dir/'swarm/swarm_cmds'
swarm_cmd_dir.mkdir(exist_ok=True, parents=True)
swarm_log_dir = liston_dir/'swarm/swarm_log'
swarm_log_dir.mkdir(exist_ok=True, parents=True)

In [46]:
nthreads = 20
cmds = []
for sub in  subjects:
    cmd = [
        'bash',
        anat_script_path,
        liston_dir.as_posix(),
        f'sub-{sub}',
        f'{nthreads}'
    ]
    cmds.append(' '.join(cmd))

In [47]:
cmds

['bash /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/anat_highres_HCP_wrapper_par.sh /data/EDB/TMSpilot/liston sub-24546 20']

In [48]:
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'liston_anat'
    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'liston_anat'
    jobid = ! swarm -f {swarm_cmd_file} -g 80 -t 22 --gres=lscratch:200 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition norm
    print(jobid)

['12206781']


## check hires outputs

In [31]:
anat_success = []
for sub in subjects:
    row = dict(subject=sub)
    for fn in ['PreFreeSurfer', 'FreeSurfer', 'PostFreeSurfer']:
        ff = liston_dir / f'sub-{sub}/anat/qa/{fn}.txt'
        flog = ff.read_text()
        row[fn] = 'Completed!' in flog.split('\n')[-2]
    anat_success.append(row)
anat_success = pd.DataFrame(anat_success)

In [32]:
anat_success

,subject,PreFreeSurfer,FreeSurfer,PostFreeSurfer
0,24563,True,True,True
1,24573,True,True,True
2,24704,True,True,True
3,24718,True,True,True
4,24740,True,True,True
5,247183,True,True,True


# Run simnibs headmodel
This takes about 4 hours and depends on the HIRES HCP Wrapper completimng

In [ ]:
outdir = Path('/data/EDB/TMSpilot/derivatives/contarg_liston_ohbm2024')
outdir.mkdir(exist_ok=True)
anat_dir = outdir / 'anat_preproc'

# build headmodels
cmds = []
for hmsub in subjects:
    ## WATCH OUT, session 1 is hardcoded!!
    session = 1
    ses_out_dir = anat_dir /f'sub-{hmsub}'
    ses_out_dir.mkdir(exist_ok=True, parents=True)
    proced_anat_dir = liston_dir / f'sub-{hmsub}/anat'
    t1w_path = proced_anat_dir / 'T1w/T1w_acpc_dc_restore.nii.gz'
    t2w_path = proced_anat_dir / 'T1w/T2w_acpc_dc_restore.nii.gz'
    hm_outdir = ses_out_dir / f"HeadModel/m2m_{hmsub}"
    if (not hm_outdir.exists()) or overwrite:
        cmd = [
            "contarg",
            "tans",
            "headmodel",
            "--t1",
            t1w_path.as_posix(),
            "--t2",
            t2w_path.as_posix(),
            "--out-dir",
            ses_out_dir.as_posix(),
            "--subject",
            hmsub
        ]
        cmds.append(' '.join(cmd))

In [ ]:
cmds

In [ ]:
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'headmodels'

    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'headmodels'
    jobids[run_name] = ! swarm -f {swarm_cmd_file} -g 80 -t 22 --module matlab,freesurfer/6,fsl,simnibs/4.0,connectome-workbench,ANTs  --time 12:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition norm
    print(jobids[run_name])

# run func_preproc+denoise
This takes about 4 hours and depends on the hires HCP wrapper, but not the headmodel.
Example command:
`./func_preproc+denoise_ME-fMRI_wrapper.sh /data/EDB/TMSpilot/liston sub-24704 15`

In [53]:
func_script_path = (liston_scripts_dir / 'func_preproc+denoise_ME-fMRI_wrapper.sh').as_posix()


swarm_cmd_dir = liston_dir/'swarm/swarm_cmds'
swarm_cmd_dir.mkdir(exist_ok=True, parents=True)
swarm_log_dir = liston_dir/'swarm/swarm_log'
swarm_log_dir.mkdir(exist_ok=True, parents=True)

In [59]:
nthreads = 20
cmds = []
for sub in ['24546']:
    cmd = [
        'bash',
        func_script_path,
        liston_dir.as_posix(),
        f'sub-{sub}',
        f'{nthreads}'
    ]
    cmds.append(' '.join(cmd))

In [60]:
cmds

['bash /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/func_preproc+denoise_ME-fMRI_wrapper.sh /data/EDB/TMSpilot/liston sub-24546 20']

In [61]:
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'liston_func'
    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'liston_func'
    jobid = ! swarm -f {swarm_cmd_file} -g 250 -t 22 --gres=lscratch:200 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition norm
    print(jobid)

['12532787']


### smooth the data and do subcortical regression
This is pretty quick (< 1 hour) and depends on the func_preproc+denoise

In [62]:
smooth_script_path = (liston_scripts_dir /'func_smooth_subcort_concat.sh').as_posix()

In [63]:
nthreads = 20
cmds = []
for sub in ['24546']:
    cmd = [
        'bash',
        smooth_script_path,
        liston_scripts_dir.as_posix(),
        f'sub-{sub}',
        liston_dir.as_posix(),
    ]
    cmds.append(' '.join(cmd))

In [28]:
cmds

['bash /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/func_smooth_subcort_concat.sh /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline sub-24718 /data/EDB/TMSpilot/liston']

In [64]:
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'liston_smooth'
    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'liston_smooth'
    jobid = ! swarm -f {swarm_cmd_file} -g 140 -t 22 --gres=lscratch:200 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 4:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition norm,quick
    print(jobid)

['12607103']


 ## split sessions

In [2]:
split_subjs = ['247183', '24740', '24742', '245461', '245462']

In [4]:
subjects = ['24546', '24563', '24573', '24704', '24718', '24740', '24742', '247183', '245461', '245462']

In [8]:
jobs = []
for subject in subjects: #['24563', '24573', '24704', '24718', '24740', '24742']:
    sub_concat_dir = liston_dir / f'sub-{subject}/func/rest/ConcatenatedCiftis'
    fd = sio.loadmat(sub_concat_dir / 'FD.mat')['FD'].squeeze()
    scanid = sio.loadmat(sub_concat_dir / 'ScanIdx.mat')['ScanIdx'].squeeze()
    scanid = pd.DataFrame(scanid, columns=['session', 'run'])
    scanid['ses_run'] = scanid.session.astype(str) + scanid.run.astype(str)
    run_starts = scanid.ses_run.drop_duplicates().index.values
    dummy_idxs = np.hstack([run_starts, run_starts + 1, run_starts +2, run_starts + 3])
    fd_mask = fd < fd_thresh
    # drop first 4 frames from each run
    fd_mask[dummy_idxs] = False
    scanid['fd_mask'] = fd_mask

    for smoothing in [2.55, 5]:
        for nruns in ['all', 2]:
            concat_nii =  sub_concat_dir / f'Rest_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing{smoothing}.dtseries.nii'
        
        for session in [1,2]:
            for nruns in ['all']:
            concat_nii =  sub_concat_dir / f'Rest_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing{smoothing}.dtseries.nii'

        for session in [1]:
            ses_concat_dir = sub_concat_dir.parent / f'session_{session}/concatenated'
            ses_concat_dir.mkdir(exist_ok=True, parents=True)
            if nruns == 'all':
                ses_concat_name = concat_nii.parts[-1].replace('Rest_', f'Rest_session-{session}_')
            elif nruns == 2:
                ses_concat_name = concat_nii.parts[-1].replace('Rest_', f'Rest_session-{session}_2runs+')
            ses_concat_out = ses_concat_dir / ses_concat_name
            cmd = [
                'wb_command',
                '-cifti-merge',
                ses_concat_out.as_posix(),
                '-cifti',
                concat_nii.as_posix(),
            ]
            if nruns == 'all':
                if subject in split_subjs:
                    if session == 1:
                        cmd += (' '.join([f'-column {trn + 1}' for trn in scanid.loc[(scanid.run < 4) & scanid.fd_mask].index.values])).split(' ')
                    elif session == 2:
                        cmd += (' '.join([f'-column {trn + 1}' for trn in scanid.loc[(scanid.run >= 4) & scanid.fd_mask].index.values])).split(' ')
                else:
                    cmd += (' '.join([f'-column {trn + 1}' for trn in scanid.loc[(scanid.session == session) & scanid.fd_mask].index.values])).split(' ')


            elif nruns == 2:
                if subject in split_subjs:
                    if session == 1:
                        cmd += (' '.join([f'-column {trn + 1}' for trn in scanid.loc[(scanid.run <= 2) & scanid.fd_mask].index.values])).split(' ')
                    elif session == 2:
                        cmd += (' '.join([f'-column {trn + 1}' for trn in scanid.loc[(scanid.run >= 4) & scanid.fd_mask & (scanid.run < 6)].index.values])).split(' ')
                else:
                    cmd += (' '.join([f'-column {trn + 1}' for trn in scanid.loc[(scanid.session == session) & scanid.fd_mask & (scanid.run <= 2)].index.values])).split(' ')
            
            jobs.append(delayed(subprocess.run)(cmd))


In [10]:
Parallel(n_jobs=20, verbose=10)(jobs)

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    9.3s
[Parallel(n_jobs=20)]: Done   6 out of  40 | elapsed:   10.3s remaining:   58.5s
[Parallel(n_jobs=20)]: Done  11 out of  40 | elapsed:   15.2s remaining:   40.2s
[Parallel(n_jobs=20)]: Done  16 out of  40 | elapsed:   15.8s remaining:   23.6s
[Parallel(n_jobs=20)]: Done  21 out of  40 | elapsed:   25.2s remaining:   22.8s
[Parallel(n_jobs=20)]: Done  26 out of  40 | elapsed:   27.1s remaining:   14.6s
[Parallel(n_jobs=20)]: Done  31 out of  40 | elapsed:   30.1s remaining:    8.7s
[Parallel(n_jobs=20)]: Done  36 out of  40 | elapsed:   36.9s remaining:    4.1s
[Parallel(n_jobs=20)]: Done  40 out of  40 | elapsed:   40.8s finished


[CompletedProcess(args=['wb_command', '-cifti-merge', '/data/EDB/TMSpilot/liston/sub-24546/func/rest/session_1/concatenated/Rest_session-1_2runs+OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing2.55.dtseries.nii', '-cifti', '/data/EDB/TMSpilot/liston/sub-24546/func/rest/ConcatenatedCiftis/Rest_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing2.55.dtseries.nii', '-column', '5', '-column', '6', '-column', '7', '-column', '8', '-column', '9', '-column', '10', '-column', '11', '-column', '12', '-column', '13', '-column', '14', '-column', '15', '-column', '16', '-column', '17', '-column', '18', '-column', '19', '-column', '20', '-column', '21', '-column', '22', '-column', '23', '-column', '24', '-column', '25', '-column', '26', '-column', '27', '-column', '28', '-column', '29', '-column', '30', '-column', '31', '-column', '32', '-column', '33', '-column', '34', '-column', '35', '-column', '36', '-column', '37', '-column', '38', '-column', '39', '-column', '40', '